In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
standings_url="https://fbref.com/en/comps/20/2016-2017/2016-2017-Bundesliga-Stats"
years = list(range(2017, 2013, -1))
all_matches = []

In [2]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0]
        Scores_Fixtures = Scores_Fixtures[['Date','Time','Comp','Round','Day','Venue','GF','GA','Opponent','Poss']]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and 'all_comps/keeper/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
        goalkeeping.columns = goalkeeping.columns.droplevel()

        #try:
        team_data = Scores_Fixtures.merge(shooting[['Date','Sh']], on = ['Date'])
        team_data = team_data.merge(goalkeeping[['Date','Save%']], on = ['Date'])

        #except ValueError:
            #continue
        team_data = team_data[team_data["Comp"] == "Bundesliga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(20)

C:\Users\tramn\AppData\Local\Temp\ipykernel_30292\3532807050.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\tramn\AppData\Local\Temp\ipykernel_30292\3532807050.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\tramn\AppData\Local\Temp\ipykernel_30292\3532807050.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
C:\Users\tramn\AppData\Local\Temp\ipykernel_30292\3532807050.py:17: FutureWarning: Passing li

In [3]:
len(all_matches)

72

In [4]:
match_df = pd.concat(all_matches)

In [6]:
match_df.columns = [c.lower() for c in match_df.columns]

In [7]:
match_df

,date,time,comp,round,day,venue,gf,ga,opponent,poss,sh,save%,season,team
2,2016-08-26,20:30,Bundesliga,Matchweek 1,Fri,Home,6,0,Werder Bremen,71.0,28.0,100.0,2017,Bayern Munich
3,2016-09-09,20:30,Bundesliga,Matchweek 2,Fri,Away,2,0,Schalke 04,69.0,18.0,100.0,2017,Bayern Munich
5,2016-09-17,15:30,Bundesliga,Matchweek 3,Sat,Home,3,1,Ingolstadt 04,69.0,10.0,80.0,2017,Bayern Munich
6,2016-09-21,20:00,Bundesliga,Matchweek 4,Wed,Home,3,0,Hertha BSC,76.0,16.0,100.0,2017,Bayern Munich
7,2016-09-24,15:30,Bundesliga,Matchweek 5,Sat,Away,1,0,Hamburger SV,69.0,17.0,100.0,2017,Bayern Munich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,2014-04-12,NaN,Bundesliga,Matchweek 30,Sat,Away,0,2,Freiburg,NaN,NaN,NaN,2014,Eintracht Braunschweig
30,2014-04-19,NaN,Bundesliga,Matchweek 31,Sat,Home,0,2,Bayern Munich,NaN,NaN,NaN,2014,Eintracht Braunschweig
31,2014-04-26,NaN,Bundesliga,Matchweek 32,Sat,Away,0,2,Hertha BSC,NaN,NaN,NaN,2014,Eintracht Braunschweig
32,2014-05-03,NaN,Bundesliga,Matchweek 33,Sat,Home,0,1,Augsburg,NaN,NaN,NaN,2014,Eintracht Braunschweig


In [8]:
match_df.to_csv("13_17_matches.csv")